In [1]:
from bestreads.models import cnn
import pandas as pd
from tensorflow import keras
import tensorflow as tf

# Generate Training and Test Dataset Split

In [2]:
data = pd.read_csv('./data/goodreads_books.csv')

cnn.split_train_val_test_data(data, test_frac=0.2, val_frac=0.2)

 26%|█████████▏                          | 13407/52199 [00:34<01:32, 421.11it/s]

./data/covers/12136160.jpg unreadable.


 63%|██████████████████████▌             | 32742/52199 [01:22<00:52, 371.42it/s]

./data/covers/00828045.jpg unreadable.


 79%|████████████████████████████▎       | 41102/52199 [01:45<00:28, 387.54it/s]

./data/covers/01059928.jpg unreadable.


100%|████████████████████████████████████| 52199/52199 [02:19<00:00, 373.81it/s]
/home/adam/miniconda3/envs/Research/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


## Create the Model

In [3]:
model = cnn.build_cnn()
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 448, 298, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 74, 64)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 110, 72, 64)       36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 18, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 16, 32)        18464     
_________________________________________________________________
flatten (Flatten)            (None, 12800)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                8

2022-05-27 11:59:06.747818: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-27 11:59:06.747984: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-27 11:59:06.748665: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


## Split Train and Validation Datasets

In [4]:
# Create the tensorflow datasets
batch_size=32

train_data = pd.read_csv('./data/processed/cnn/train_ratings.csv')
val_data = pd.read_csv('./data/processed/cnn/val_ratings.csv')

# Shuffle the data
train_data = train_data.sample(frac=1, random_state=2974)
val_data = val_data.sample(frac=1, random_state=2143)

# Create Tensorflow datasets
train_dataset = cnn.create_dataset(train_data.file_path,
                                   train_data.average_rating,
                                   batch_size=batch_size)
val_dataset = cnn.create_dataset(val_data.file_path,
                                 val_data.average_rating,
                                 batch_size=batch_size)

## Train Model

In [ ]:
print(tf.test.is_gpu_available())
history = model.fit(train_dataset, epochs=10, verbose=1, validation_data=val_dataset)

## Save model

In [ ]:
cnn.save_model(model, save_dir = './models/')

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])